In [ ]:
import os
import pandas as pd
from glob import glob
from collections import defaultdict

# === CONFIGURACIÓN ===
base_dir = r'C:\Users\Felipe Carriel\Documents\Brain\High-Order-interactions\Ongoing_EEG_Source_analysis_Loreta_para_redes'
grupos = ['CN', 'AD', 'DFT']
data = {}

for grupo in grupos:
    print(f"\n🔍 Procesando grupo: {grupo}")
    group_dir = os.path.join(base_dir, f"{grupo}_HOI")
    
    group_data = defaultdict(lambda: defaultdict(lambda: {'sinergia': {}, 'redundancia': {}}))

    # Recorre todas las subcarpetas (red/banda)
    for root, _, files in os.walk(group_dir):
        for file in files:
            if file.endswith('_Oinfo.csv'):
                filepath = os.path.join(root, file)
                rel_path = os.path.relpath(filepath, group_dir)
                parts = rel_path.split(os.sep)

                if len(parts) < 3:
                    print(f"⚠️ Ruta inesperada: {rel_path}")
                    continue

                red = parts[0]
                banda = parts[1]
                filename = parts[2]

                # Extraer sujeto y orden desde el nombre del archivo
                try:
                    name_parts = filename.split('_')
                    subject_id = '_'.join(name_parts[:3])  # ej: subj1_DMN_n3
                    n_str = [p for p in name_parts if p.startswith('n') and p[1:].isdigit()]
                    if not n_str:
                        print(f"⚠️ No se encontró 'n' en el nombre: {filename}")
                        continue
                    order = int(n_str[0][1:])  # 'n3' -> 3
                except Exception as e:
                    print(f"⚠️ Error al extraer info desde: {filename}, {e}")
                    continue

                # Leer el archivo y calcular métricas
                try:
                    df = pd.read_csv(filepath, header=None)
                    values = df[0].dropna()

                    sinergia_vals = values[values < 0]
                    redundancia_vals = values[values > 0]

                    sinergia_mean = sinergia_vals.mean() if not sinergia_vals.empty else 0
                    redundancia_mean = redundancia_vals.mean() if not redundancia_vals.empty else 0

                    group_data[subject_id][f"{red}_{banda}"]['sinergia'][order] = sinergia_mean
                    group_data[subject_id][f"{red}_{banda}"]['redundancia'][order] = redundancia_mean

                except Exception as e:
                    print(f"⚠️ Error leyendo {filename}: {e}")
                    continue

    data[grupo] = dict(group_data)

print("\n✅ Estructura de datos generada.")


In [ ]:
def data_to_dataframe(data, tipo='sinergia'):
    rows = []
    for grupo, sujetos in data.items():
        for sujeto, redes in sujetos.items():
            row = {'Grupo': grupo, 'Sujeto': sujeto}
            for red_banda, metrics in redes.items():
                for orden, valor in metrics[tipo].items():
                    col_name = f"{tipo}_{red_banda}_n{orden}"
                    row[col_name] = valor
            rows.append(row)
    return pd.DataFrame(rows)

# DataFrames originales sin escalar
df_sinergia = data_to_dataframe(data, tipo='sinergia')
df_redundancia = data_to_dataframe(data, tipo='redundancia')

# Separar por grupo
df_sinergia_CN  = df_sinergia[df_sinergia['Grupo'] == 'CN'].reset_index(drop=True)
df_sinergia_AD  = df_sinergia[df_sinergia['Grupo'] == 'AD'].reset_index(drop=True)
df_sinergia_DFT = df_sinergia[df_sinergia['Grupo'] == 'DFT'].reset_index(drop=True)

df_redundancia_CN  = df_redundancia[df_redundancia['Grupo'] == 'CN'].reset_index(drop=True)
df_redundancia_AD  = df_redundancia[df_redundancia['Grupo'] == 'AD'].reset_index(drop=True)
df_redundancia_DFT = df_redundancia[df_redundancia['Grupo'] == 'DFT'].reset_index(drop=True)


In [ ]:
import pandas as pd

df_sinergia = pd.read_excel("HOI_por_grupo.xlsx", sheet_name='Sinergia_Todos')
df_sinergia_CN = pd.read_excel("HOI_por_grupo.xlsx", sheet_name='Sinergia_CN')
df_sinergia_AD = pd.read_excel("HOI_por_grupo.xlsx", sheet_name='Sinergia_AD')
df_sinergia_DFT = pd.read_excel("HOI_por_grupo.xlsx", sheet_name='Sinergia_DFT')

df_redundancia = pd.read_excel("HOI_por_grupo.xlsx", sheet_name='Redundancia_Todos')
df_redundancia_CN = pd.read_excel("HOI_por_grupo.xlsx", sheet_name='Redundancia_CN')
df_redundancia_AD = pd.read_excel("HOI_por_grupo.xlsx", sheet_name='Redundancia_AD')
df_redundancia_DFT = pd.read_excel("HOI_por_grupo.xlsx", sheet_name='Redundancia_DFT')


In [ ]:
import pandas as pd

# Cargar todos los DataFrames
sheets = {
    'Sinergia_Todos': pd.read_excel("HOI_por_grupo.xlsx", sheet_name='Sinergia_Todos'),
    'Sinergia_CN': pd.read_excel("HOI_por_grupo.xlsx", sheet_name='Sinergia_CN'),
    'Sinergia_AD': pd.read_excel("HOI_por_grupo.xlsx", sheet_name='Sinergia_AD'),
    'Sinergia_DFT': pd.read_excel("HOI_por_grupo.xlsx", sheet_name='Sinergia_DFT'),
    'Redundancia_Todos': pd.read_excel("HOI_por_grupo.xlsx", sheet_name='Redundancia_Todos'),
    'Redundancia_CN': pd.read_excel("HOI_por_grupo.xlsx", sheet_name='Redundancia_CN'),
    'Redundancia_AD': pd.read_excel("HOI_por_grupo.xlsx", sheet_name='Redundancia_AD'),
    'Redundancia_DFT': pd.read_excel("HOI_por_grupo.xlsx", sheet_name='Redundancia_DFT'),
}

# Diccionario de reemplazo
redes_dict = {
    "auditory": "AUDITORY",
    "default": "DEFAULT",
    "dmn_anterior": "DMNAnterior",
    "dmn_medial_temporal": "DMNMedialTemporal",
    "dmn_posterior": "DMNPosterior",
    "dorsal_attention": "DORSALATTENTION_DAN_",
    "fpcn_left": "FPCNLeft",
    "fpcn_right": "FPCNRight",
    "fpcn_ecn": "FPCN_ECN_",
    "language": "LANGUAGE",
    "limbic": "LIMBIC",
    "medial_temporal_lobe": "MedialTemporalLobe",
    "posterior_multimodal": "PosteriorMultimodal",
    "salience": "SALIENCE",
    "somatosensory": "SOMATOSENSORY",
    "ventral_attention": "VENTRALATTENTION_VAN_",
    "visual": "VISUAL",
    "visual_association": "VisualAssociation"
}

# Invertimos el diccionario: claves = texto actual, valores = texto deseado
redes_dict_invertido = {v: k for k, v in redes_dict.items()}

# Función para renombrar columnas
def renombrar_columnas(df):
    nuevas_columnas = []
    for col in df.columns:
        nueva_col = col
        for actual, nuevo in redes_dict_invertido.items():
            if actual in col:
                nueva_col = col.replace(actual, nuevo)
                break
        nuevas_columnas.append(nueva_col)
    df.columns = nuevas_columnas
    return df

# Renombrar columnas en todos los DataFrames
sheets_renombrados = {name: renombrar_columnas(df.copy()) for name, df in sheets.items()}

# Guardar nuevo archivo Excel
with pd.ExcelWriter("HOI_por_grupo_v2.xlsx") as writer:
    for sheet_name, df in sheets_renombrados.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print("✅ Archivo 'HOI_por_grupo_v2.xlsx' creado correctamente.")


In [5]:
import pandas as pd

# Leer cada hoja del archivo v2
df_sinergia = pd.read_excel("HOI_por_grupo_v2.xlsx", sheet_name='Sinergia_Todos')
df_sinergia_CN = pd.read_excel("HOI_por_grupo_v2.xlsx", sheet_name='Sinergia_CN')
df_sinergia_AD = pd.read_excel("HOI_por_grupo_v2.xlsx", sheet_name='Sinergia_AD')
df_sinergia_DFT = pd.read_excel("HOI_por_grupo_v2.xlsx", sheet_name='Sinergia_DFT')

df_redundancia = pd.read_excel("HOI_por_grupo_v2.xlsx", sheet_name='Redundancia_Todos')
df_redundancia_CN = pd.read_excel("HOI_por_grupo_v2.xlsx", sheet_name='Redundancia_CN')
df_redundancia_AD = pd.read_excel("HOI_por_grupo_v2.xlsx", sheet_name='Redundancia_AD')
df_redundancia_DFT = pd.read_excel("HOI_por_grupo_v2.xlsx", sheet_name='Redundancia_DFT')
